# Use Case: Reliability of the LULC classification

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define settings

In [ ]:
# Time
year_start = 2019
year_stop = 2020

# Region of interest
lon_slice = slice(-10, 0)
lat_slice = slice(45, 35)

# Define IPCC dict
ipcc_dict = {
    "No Data": ("#000000", [0]),
    "Agriculture": ("#ffffcc", [10, 11, 12, 20, 30, 40]),
    "Forest": ("#4c9900", [50, 60, 61, 62, 70, 71, 72, 80, 81, 82, 90, 100, 160, 170]),
    "Grassland": ("#ccff99", [110, 130]),
    "Settlement": ("#ff0000", [190]),
    "Wetland": ("#99ffff", [180]),
    "Other": ("#0000ff", [120, 121, 122, 140, 150, 151, 152, 153, 200, 201, 202, 210]),
}

## Download data

In [ ]:
# Download and transform (cached)
request = (
    "satellite-land-cover",
    {
        "variable": "all",
        "format": "zip",
        "version": "v2.1.1",
        "year": [str(year) for year in range(year_start, year_stop + 1)],
    },
)
ds = download.download_and_transform(
    *request,
    chunks={"year": 1},
    transform_func=utils.regionalise,
    transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
)

# Rename time dimension
ds["time"] = ds["time"].dt.year
ds = ds.rename(time="year")

# Maps

In [ ]:
# Plot multiple years, auto infer colors and legend
plot.lccs_map(ds["lccs_class"], row="year", interpolation="antialiased")

In [ ]:
# Plot single year, specifying color and legend
plot.lccs_map(
    ds["lccs_class"].sel(year=2020).squeeze(),
    legend_dict=ipcc_dict,
    interpolation="antialiased",
)

## Bars

In [ ]:
# Plot multiple years, auto infer colors and labels
plot.lccs_bar(
    ds["observation_count"],
    ds["lccs_class"],
    title=f"years: {ds['year'].values.tolist()}",
)

In [ ]:
# Plot single year, specifying color and legend
year = 2020
ds_year = ds.sel(year=year)
plot.lccs_bar(
    ds_year["observation_count"],
    ds_year["lccs_class"],
    ipcc_dict,
    xlabel="IPCC Groups",
    title=f"years: {ds_year['year'].values.tolist()}",
)

## Binned bars

In [ ]:
# Plot sum grouped by latitude
groupby_dim = "latitude"
groupby_bins = range(
    int(ds_year[groupby_dim].min() // 1), int(ds_year[groupby_dim].max() // 1) + 2
)
plot.lccs_bar(
    ds_year["observation_count"],
    ds_year["lccs_class"],
    ipcc_dict,
    reduction="sum",
    groupby_bins_dims={groupby_dim: groupby_bins},
    title=f"years: {ds_year['year'].values.tolist()}",
    stacked=True,
)

In [ ]:
# Plot sum grouped by longitude, focus on water bodies
groupby_dim = "longitude"
groupby_bins = range(
    int(ds_year[groupby_dim].min() // 1), int(ds_year[groupby_dim].max() // 1) + 2
)

water_flag = 210
all_flags = list(
    range(
        ds_year["lccs_class"].attrs["valid_min"],
        ds_year["lccs_class"].attrs["valid_max"] + 1,
    )
)
all_flags.remove(water_flag)

labels_dict = {"Water bodies": ("b", water_flag), "All Others": ("g", all_flags)}
plot.lccs_bar(
    ds_year["observation_count"],
    ds_year["lccs_class"],
    labels_dict,
    reduction="sum",
    groupby_bins_dims={groupby_dim: groupby_bins},
    title=f"years: {ds_year['year'].values.tolist()}",
    stacked=True,
)